In [1]:
import warnings
warnings.filterwarnings('ignore')

import itertools

import os
from glob import glob
import pickle

import numpy as np
import pandas as pd

import nltk
import re
import codecs
import unidecode
#pip install unidecode
import mpld3
# pip install mpld3
import stop_words
# pip install stop-words
from nltk import SnowballStemmer, pos_tag, word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import HashingVectorizer,TfidfTransformer,TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import *
from sklearn.linear_model import LogisticRegression
from sklearn.svm import *
from sklearn.semi_supervised import *

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#download cv in a list
def load_cv_list():
    path = '../Commun/Data_talan/txt/'
    filenames = [path+directory for directory in os.listdir(path)]
    liste_cv = []
    for file in filenames:
        liste_cv.append(open(file).read())
    return liste_cv,filenames

In [3]:
liste_cv, _ =  load_cv_list()

In [4]:
len(liste_cv)

632

In [5]:
import string, re
def del_line_feed(s):  
    """Delete \n in the text"""
    regex = re.compile('[%s]' % '(\\n)*(\\x0c)*(\\uf0a7)*')
    return regex.sub(' ', s)

In [6]:
string.punctuation
def del_punct(s):  
    """Delete punctuation in the text"""
    #le maintient de la ponctuation pertube le stop words, apostrophe gérée dans text_treatment
    regex = re.compile('[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_{|}~')) 
    return regex.sub(' ', s)

In [7]:
liste_cv = [del_punct(del_line_feed(cv)) for cv in liste_cv]

In [8]:
def _calculate_languages_ratios(text):
    """
    Calculate probability of given text to be written in several languages and
    return a dictionary that looks like {'french': 2, 'spanish': 4, 'english': 0}
    """

    languages_ratios = {}

    '''
    nltk.wordpunct_tokenize() splits all punctuations into separate tokens
    
    >>> wordpunct_tokenize("That's thirty minutes away. I'll be there in ten.")
    ['That', "'", 's', 'thirty', 'minutes', 'away', '.', 'I', "'", 'll', 'be', 'there', 'in', 'ten', '.']
    '''

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens] #from text get list of word in minuscule

    
    for language in stopwords.fileids(): # pour chaque langue
        stopwords_set = set(stopwords.words(language)) #je mets les stop words du langage dans un set
        words_set = set(words) #je mets les mots de mon texte dans un set
        #je prends l'intersection entre les mots de mon texte et les mots du stopwords dans le langage donné
        common_elements = words_set & stopwords_set
        
        #je compute mon score comme le nombre d'éléments en communs dictionnaire [langage : score]
        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios

In [9]:
import nltk
nltk.download('stopwords')
stopwords.fileids()

def get_cv_langue(liste_cv) :
    """Return resume witten in the specified language in parameter"""
    liste_2 = []
    i=0
    for cv in liste_cv:
        if max(_calculate_languages_ratios(cv),key =_calculate_languages_ratios(cv).get)=='french':
            liste_2.append(cv)
        i+=1
    return liste_2

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mehdiregina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
liste_cv = get_cv_langue(liste_cv) 

In [11]:
def text_treatment (text):
    text = text.lower()
    text = text.replace("\x00", '').replace("\x01", '').replace("\x02", '').replace("\x03", '') \
    .replace("\x04", '').replace("\x05", '').replace("\x06", '').replace("\x07", '').replace("\x08", '') \
    .replace("\x0e", '').replace("\x11", '').replace("\x12", '').replace("\x10", '').replace("\x19", '') \
    .replace("\x1b", '').replace("\x14", '').replace("\x15", '').replace('/', '').replace('=', '').replace("〓", "") \
    .replace("»", "").replace("«", "").replace("¬", "").replace('`', '').replace (" -", "").replace("•", "")\
    .replace("l'", "").replace("l’", "").replace("l´", "").replace("d’", "").replace("d'", "").replace("d´","")\
    .replace("j’", "").replace("j'", "").replace("j´","").replace("n’", "").replace("n'", "").replace("n´","")\
    .replace("”", "").replace("~", "").replace("§", "").replace("¨", "").replace("©", "").replace("›", "")\
    .replace("₋", "").replace("→", "").replace("⇨", "").replace("∎", "").replace("√", "").replace("□", "")\
    .replace("*", "").replace("&", "").replace("►", "").replace("◊", "").replace("☞", "").replace("#", "")\
    .replace("%", "").replace("❖", "").replace("➠", "").replace("➢", "").replace("", "").replace("✓", "") \
    .replace("√", "").replace("✔", "").replace("♦", "").replace("◦", "").replace("●", "").replace("▫", "")\
    .replace("▪", "").replace("…", "").replace("þ", "").replace("®", "").replace('', '').replace("...", "")
    text = unidecode.unidecode(text) # remove accent
    return text

La liste de stop words importe peu connaissant le vocabulaire à retenir pour le count vectorizer

In [12]:
#generate stopwords
stop_words_py = set(stop_words.get_stop_words('french'))

# attention certains stop words pourraient être utiles par la suite
stopwords_set_manuel = set(["an", "ans", 'les', 'moins', 'd\'un','janvier', 'fevrier', 'février', 'mars', 'avril', \
                 'mai', 'juin', 'juillet', 'aout', 'août', 'septembre', 'octobre', 'novembre', 'décembre', \
                  'decembre', 'moins', 'mise', 'universit\xc3\xa9', 'université', 'universite', 'ion','sage', \
                  'o', 'rac', 'vers', 'via', 'p\xc3\xa9rim\xc3\xa8tre', 'périmètre','et','paris','x',"\x00",\
                          "\x01","\x02", "\x03","\x04","\x05","\x06","\x07","\x08","\x09","\x0e","\x0e","\x11",\
                           "\x12","\x13","\x14","\x15","\x16","\x17","\x18","\x19",\
                           "guide","10g","11g","9i",'ad','v10','v2','v3','v5','v6','v8','v9',])
stop_words_main = stop_words_py | stopwords_set_manuel
stop_words_main = list(stop_words_main)
print("taille stop words liste : ", len(stop_words_main))

taille stop words liste :  311


In [13]:
def remove_stopwords(text,stopwords_list):
    text_temp = " ".join(text.split())+" "
    for word in stopwords_list:
        text_temp = text_temp.replace(" "+word+" ", " ")
    return text_temp

In [14]:
liste_cv = [remove_stopwords(text_treatment(cv),stop_words_main) for cv in liste_cv]

In [15]:
#SnowballStemmer 
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters 
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters 
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [16]:
liste_cv_tokenized = [tokenize_only(cv) for cv in liste_cv]
liste_cv_tokenized[0]

['consultant',
 'junior',
 'amoa',
 'formation',
 'master',
 'entrepreneuriat',
 'strategie',
 'marketing',
 'commerciale',
 'elaboration',
 'business',
 'plan',
 'gestion',
 'financiere',
 "sup'career",
 'master',
 'entrepreneuriat',
 'conception',
 'evaluation',
 'projets',
 'management',
 'organisations',
 'controle',
 'gestion',
 'creteil',
 'competences',
 'savoir',
 'gestion',
 'relation',
 'client',
 'conseil',
 'gestion',
 'portefeuille',
 'clients',
 'professionnels',
 'techniques',
 'planification',
 'gestion',
 'projets',
 'marketing',
 'informatique',
 'logiciels',
 'gestion',
 'erp',
 'crm',
 'bureautique',
 'pack',
 'office',
 'word',
 'excel',
 'access',
 'powerpoint',
 'experience',
 'professionnelle',
 'teleconseillere',
 'qualigaz',
 'aubervilliers',
 'conseillere',
 'clientele',
 'comearth',
 'orsay',
 'assistante',
 'maternelle',
 'particulier',
 'conseillere',
 'clientele',
 'olympus',
 'france',
 'rungis',
 'assistante',
 'administrative',
 'mtpro',
 'services',
 

load vocabulary

In [17]:
#vocab116
with open("vocabcv116.txt","r") as file:
    vocab_list = file.read()
vocab116_list = vocab_list.split(",")[:-1]

In [18]:
#vocab204
with open("vocabcv204.txt","r") as file:
    vocab_list = file.read()
vocab204_list = vocab_list.split(",")[:-1]

Count Vectorizer

In [19]:
#116
count116_vec = CountVectorizer(stop_words=stop_words_main,vocabulary=vocab116_list,\
                           preprocessor=text_treatment,tokenizer=tokenize_only)

count116_cv = count116_vec.fit_transform(liste_cv)

In [20]:
#representation 116
vocab_count116 = count116_vec.vocabulary

In [21]:
def get_representation(count,count_vocab):
    liste_cv = []
    for i,cv_row in enumerate(count.toarray()):
        liste_row = []
        for j,elt in enumerate(cv_row):
            if elt>0:
                liste_row.append(count_vocab[j])
        liste_cv.append(liste_row)
    
    return liste_cv

In [22]:
cv_word116 = get_representation(count116_cv,vocab_count116)

save representation

In [23]:
#save representation
thefile = open('testarm.txt', 'w')
for row in cv_word116:
    for word in row:
        thefile.write(word+',')
    thefile.write('\n')

load rules

In [24]:
#vocab116
with open("rules2075.txt","r") as file:
    rules2075 = file.read()
#_list = vocab_list.split(",")[:-1]

In [25]:
#vocab116
with open("rules2085.txt","r") as file:
    rules2085 = file.read()
#vocab116_list = vocab_list.split(",")[:-1]

In [31]:
#rule to dictionnary
dic_rule2085 = {}
dic_rule2085["confidence"] = 85
for test in rules2085.split("\n"):
    inter = re.sub(r"Rule#\s*[0-9]*\s*:|[0-9]{2}\s[0-9]{2}|\[|\]|\'","",test)
    if len(inter.split(" ==> "))>1 and len(inter.split(" ==> ")[1])>1:
        key,val = inter.split(" ==> ")
        key = key.strip()
        val = val.strip()
        if key in dic_rule2085.keys():
            dic_rule2085[key].append(val)
        else:
            dic_rule2085[key] = [val]

dic_rule2075 = {}
dic_rule2075["confidence"] = 75
for test in rules2075.split("\n"):
    inter = re.sub(r"Rule#\s*[0-9]*\s*:|[0-9]{2}\s[0-9]{2}|\[|\]|\'","",test)
    inter.strip(" ")
    if len(inter.split(" ==> "))>1:
        key,val = inter.split(" ==> ")
        key = key.strip()
        val = val.strip()
        if key in dic_rule2075.keys():
            dic_rule2075[key].append(val)
        else:
            dic_rule2075[key] = [val]

predict missing words in CVs

combinaisons ordre 1 à 3 ...

In [32]:
cv_word116[0]

['logiciels',
 'crm',
 'management',
 'consultant',
 'techniques',
 'projets',
 'conception',
 'erp',
 'projet',
 'planification',
 'excel',
 'access',
 'formation',
 'office',
 'word',
 'amoa',
 'business',
 'gestion']

In [33]:
def pred_missing_skills(dic_rule,cv_list):
    print("------------ CONFIDENCE: {}%-------------".format(dic_rule["confidence"]))
    pred_list = []
    for cv in cv_list:
        set_existing = set(cv)
        
        #1st ordrer
        pred_1st_order_raw = []
        for skill in cv :
            if skill in dic_rule.keys():
                #keep flat list
                pred_1st_order_raw += dic_rule[skill]
        
        pred_1st_order_fin = set(pred_1st_order_raw) - set_existing
        
        #2nd order
        pred_2nd_order_raw = []
        for sub_tuples in itertools.combinations(cv,2) :
            candidate = sub_tuples[0]+", "+sub_tuples[1]
            if candidate in dic_rule.keys():
                #keep flat list
                pred_2nd_order_raw += dic_rule[candidate]
        
        pred_2nd_order_fin = set(pred_2nd_order_raw) - set_existing
        
        #3rd order
        pred_3rd_order_raw = []
        for sub_tuples in itertools.combinations(cv,3) :
            candidate = sub_tuples[0]+", "+sub_tuples[1]+", "+sub_tuples[2]
            if candidate in dic_rule.keys():
                #keep flat list
                pred_3rd_order_raw += dic_rule[candidate]
        
        pred_3rd_order_fin = set(pred_3rd_order_raw) - set_existing
        
        pred_fin = pred_1st_order_fin & pred_2nd_order_fin & pred_3rd_order_fin
        pred_list.append(list(pred_fin))
        
    return pred_list  

In [35]:
liste_pred85 = pred_missing_skills(dic_rule2085,cv_word116)

------------ CONFIDENCE: 85%-------------


In [36]:
liste_pred75 = pred_missing_skills(dic_rule2075,cv_word116)

------------ CONFIDENCE: 75%-------------


In [37]:
liste_pred75

[['donnees', 'developpement', 'sql'],
 [],
 [],
 ['techniques', 'conception'],
 ['outils', 'redaction', 'fonctionnelles'],
 ['bi', 'suivi', 'outils'],
 ['suivi', 'redaction', 'bi', 'conception'],
 ['techniques', 'formation'],
 ['outils', 'projet', 'developpement', 'oracle', 'suivi'],
 ['conception', 'analyse'],
 ['redaction', 'donnees', 'projet', 'sql', 'oracle'],
 ['techniques', 'suivi', 'fonctionnelles', 'projet'],
 ['outils', 'analyse'],
 ['sql', 'formation'],
 ['outils',
  'redaction',
  'projet',
  'sql',
  'developpement',
  'oracle',
  'suivi',
  'analyse'],
 [],
 ['redaction', 'analyse'],
 ['outils', 'redaction', 'gestion', 'sql', 'analyse'],
 ['outils', 'developpement', 'donnees'],
 ['outils'],
 ['business', 'conception', 'developpement', 'techniques'],
 ['outils'],
 ['suivi', 'conception', 'analyse'],
 ['redaction', 'fonctionnelles'],
 ['sql'],
 [],
 ['bi', 'fonctionnelles'],
 ['business', 'formation', 'sql', 'donnees'],
 ['suivi', 'fonctionnelles'],
 ['outils', 'redaction', 

In [38]:
liste_pred85

[['donnees'],
 [],
 [],
 [],
 ['redaction'],
 [],
 [],
 [],
 ['developpement', 'projet'],
 [],
 ['donnees', 'sql', 'projet'],
 ['projet'],
 [],
 ['sql'],
 ['sql', 'projet'],
 [],
 [],
 ['sql', 'gestion'],
 ['donnees', 'developpement'],
 [],
 ['developpement'],
 [],
 [],
 ['redaction'],
 ['sql'],
 [],
 [],
 ['donnees', 'formation', 'sql'],
 [],
 [],
 ['developpement', 'gestion'],
 [],
 ['specifications'],
 ['donnees', 'redaction', 'developpement', 'sql'],
 ['developpement', 'gestion'],
 [],
 [],
 ['sql', 'developpement', 'oracle'],
 ['redaction'],
 [],
 ['donnees'],
 [],
 ['donnees', 'sql', 'gestion'],
 [],
 ['redaction', 'specifications'],
 ['redaction', 'donnees', 'gestion', 'sql', 'developpement'],
 [],
 ['donnees', 'developpement'],
 ['developpement'],
 [],
 ['donnees'],
 ['sql', 'projet'],
 [],
 ['outils'],
 ['redaction'],
 [],
 ['donnees', 'redaction', 'developpement', 'specifications'],
 ['redaction', 'developpement', 'gestion'],
 [],
 ['sql', 'developpement', 'specifications'],
